# Lesson 3 - BOLDspins

The `BOLDspins` module contains the objects which define Monte Carlo spin diffusion in simulations. To begin we first import the module as follows:

In [ ]:
from boldswimsuite import BOLDgeometry, BOLDspins

## Using Continuous Voxels

We will start by randomly generating a continuous voxel, as spins require a space in which to exist.

In [ ]:
random_continuous_voxel = BOLDgeometry.ContinuousVoxel3D.from_random(
    size=3,
    CBV=0.10,
    B0=3,
    labels=['vein', 'artery'],
    weights={
        'vein':1, 
        'artery':1
    },
    diameter_distributions={
        'vein': [0.2, 0.3, 0.4], 
        'artery': [0.3, 0.4, 0.5]
    },
    dchis={
        'vein': 3e-8,
        'artery': 4e-8
    },
    permeation_probabilities={
        'vein': 0, 
        'artery': 0.1
    },
    vessel_type='cylinder',
    allow_vessel_intersection=True,
    seed=0, #repeating with the same seed with provide the same result
    progressbar=True
)

print(f'Number of vessels: {len(random_continuous_voxel.vessels)}')

Now we can create a `Spins3D` object using the default constructor, which has the following arguments:
- ADC : float, the apparent diffusion coefficient of water molecules in the tissue (mm^2/s).
- num_spins : int, the number of Monte Carlo spins to create.
- geometry : BOLDgeometry.ContinuousVoxel3D | BOLDgeometry.DiscreteVoxel3D, the voxel object on which the spins will diffuse.
- dt : float, the initial step length (ms).
- IV : bool, if False, will only position spins in the extravascular space. Default is True.
- seed : int, seed for the random number generator.

In [ ]:
spins = BOLDspins.Spins3D(
    ADC=0.001, 
    num_spins=5,
    geometry=random_continuous_voxel,
    dt=0.2,
    IV=True,
    seed=0 #repeating with the same seed with provide the same result
)

Creating the spins not only positions the spins, but also calculates the dephasing and vessel indices for each spin. We can get all of these (and the `dt` from which these were calculated) with the `get_phase_vessel_index_dt` method.

In [ ]:
phase, vessel_indices, dt = spins.get_phase_vessel_indices_dt()

print(f'Phase: {phase}')
print(f'vessel indices: {vessel_indices}')
print(f'dt: {dt}')

We can now advance the simulation by a step, which will move all the spins according to the parameters and geometry provided. We use the `step` method of the spins object, which takes the time step length `dt` as an argument. This way different steps can have different time step lengths.

In [ ]:
spins.step(dt=0.1)

Once again we use the `get_phase_vessel_indices_dt` method to retrieve the relevant information from the most recent step.

In [ ]:
phase, vessel_indices, dt = spins.get_phase_vessel_indices_dt()

print(f'Phase: {phase}')
print(f'is IV: {vessel_indices}')
print(f'dt: {dt}')

We can continue to use the `step` function to add additional Monte Carlo steps to the simulation. This is covered more in detail in the next lesson ("Lesson4_BOLDsequence.ipynb").

## Using Discrete Voxels

Using discrete voxels with the spins object is essentially the same as with continuous voxels, but we must first create a discrete voxel object:

In [ ]:
discrete_voxel = BOLDgeometry.DiscreteVoxel3D.from_continuous_analytical(
    N=100,
    voxel=random_continuous_voxel
)

We then proceed by creating a spins object but we provide the discrete voxel for the `geometry` argument.

In [ ]:
discrete_voxel_spins = BOLDspins.Spins3D(
    ADC=0.001,
    num_spins=5,
    geometry=discrete_voxel,
    dt=0.2,
    IV=True,
    seed=0 #repeating with the same seed with provide the same result
)

The spins object behaves the same way as when using the continuous voxel:

In [ ]:
phase, vessel_indices, dt = discrete_voxel_spins.get_phase_vessel_indices_dt()

print(f'Phase: {phase}')
print(f'is IV: {vessel_indices}')
print(f'dt: {dt}')

In [ ]:
discrete_voxel_spins.step(dt=0.1)

In [ ]:
phase, vessel_indices, dt = discrete_voxel_spins.get_phase_vessel_indices_dt()

print(f'Phase: {phase}')
print(f'is IV: {vessel_indices}')
print(f'dt: {dt}')